In [ ]:
import numpy as np 
import pandas as pd
import plotly.express as px 
import sklearn
import torch

In [ ]:
df_train = pd.read_csv('../input/fashion-mnist_train.csv')
df_test = pd.read_csv('../input/fashion-mnist_test.csv')
X_train = df_train.drop('label', axis=1).to_numpy().reshape((60000, 28, 28))
X_test = df_test.drop('label', axis=1).to_numpy().reshape((10000, 28, 28))
y_train = pd.get_dummies(df_train['label']).rename(columns={0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}).to_numpy()
y_test = pd.get_dummies(df_test['label']).rename(columns={0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}).to_numpy()

In [ ]:
def adjacent_pixels(pixel_cord: tuple):
    x = pixel_cord[0]
    y = pixel_cord[1]
    cord_matrix = np.array([[(max(0, x - 1), max(0, y - 1)), (max(0, x - 1), y), (max(0, x - 1), min(27, y + 1))], [(x, max(0, y - 1)), pixel_cord, (x, min(27, y + 1))], [(min(27, x + 1), max(0, y - 1)), (min(27, x + 1), y), (min(27, x + 1), min(27, y + 1))]])
    return cord_matrix
    

In [ ]:
image = X_train[0]
difference_sums = []
for x in range(1, 27):
    for y in range(1, 27):
        pixel_selection = (x, y)
        arr = adjacent_pixels(pixel_selection)
        minuend = np.full((3, 3), image[x][y])
        subtrahend = []
        for i in arr:
            for j in i:
                subtrahend.append(image[j[0]][j[1]])
        subtrahend = np.array(subtrahend).reshape((3, 3))
        difference = np.subtract(minuend, subtrahend).sum()
        difference_sums.append(difference)

print(difference_sums)

In [ ]:
fig = px.histogram(x=difference_sums)
fig.show()

In [ ]:
skewed_distribution = np.array(list(map(lambda x: x if abs(x) >= 300 else 0, difference_sums))).reshape((26, 26))

In [ ]:
px.imshow(skewed_distribution)